In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

from ase.db import connect

import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
from preprocess.distance import *
from preprocess.convert import *

In [3]:
row = train_db.get(1)

In [4]:
dist_mat, _ = get_distance_matrix(row)

In [5]:
get_bond_counts(dist_mat, row.symbols)

{'CC2': 4,
 'CH1': 19,
 'CS2': 2,
 'CN1': 8,
 'CC1': 8,
 'CO1': 2,
 'HN1': 1,
 'CN2': 2}

In [6]:
bond_matrix = get_bond_type_matrix(dist_mat, row.symbols)

bond_matrix

array([[0., 2., 0., ..., 0., 0., 0.],
       [2., 0., 2., ..., 0., 0., 0.],
       [0., 2., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
bond_matrix.sum(axis=1)

array([5., 7., 5., 6., 4., 4., 2., 3., 3., 3., 1., 3., 3., 1., 4., 5., 4.,
       6., 4., 3., 3., 3., 3., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [9]:
def display_molecule(row, plt):
    fig = plt.figure()
    fig.set_size_inches(12, 8)

    ax = plt.axes(projection='3d')
    # ax.grid(False)
    
    def init():
        p = row.positions
        s = row.symbols
        n = row.numbers
        
        x = p[:, 0]
        y = p[:, 1]
        z = p[:, 2]

        ax.scatter(x, y, z, 
            s = n*20,        # size
            c = n,           # color
            cmap = 'inferno'
        )

        dist_mat, _ = get_distance_matrix(row)
        bond_matrix = get_bond_type_matrix(dist_mat, row.symbols)

        for i in range(len(p)):
            ax.text(x[i], y[i], z[i], " " + s[i])

        for i in range(len(p)):
            for j in range(i+1, len(p)):
                if bond_matrix[i][j] != 0:
                    lx = np.array([x[i], x[j]])
                    ly = np.array([y[i], y[j]])        
                    lz = np.array([z[i], z[j]])

                    ax.plot(lx, ly, lz, color='black')
                    ax.text(lx.mean(), ly.mean(), lz.mean(), str(int(bond_matrix[i][j])))
        return fig,


    def animate(i):
        ax.view_init(elev=10., azim=i)
        return fig,

    
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=360, interval=30, blit=True)
    
    plt.close()
    return HTML(anim.to_html5_video())

display_molecule(row, plt)